In [1]:
import os
import tensorflow
from tensorflow import keras
from tensorflow.keras.models import Model
from keras.layers import Input
from keras.models import Sequential, Model
from keras.layers import Conv2D, Dense, MaxPooling2D, Flatten, Dropout, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from models import resnet_v1, resnet_v2

Using TensorFlow backend.


In [2]:
train_dir = "data/traindata"   ###训练数据路径
test_dir = "data/traindata"   ###测试数据路径

In [3]:
## Generators ##
 
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    validation_split=0.2,
)

test_datagen = ImageDataGenerator(
    rescale = 1./255,
)


In [4]:
train_dataset = train_datagen.flow_from_directory(
    train_dir,
    #class_mode='binary',
    class_mode='categorical',
    batch_size = 2,
    shuffle=True,
    subset = "training",
    target_size = (260, 1323)
)

valid_dataset = train_datagen.flow_from_directory(
    train_dir,
    #class_mode='binary',
    class_mode='categorical',
    batch_size = 2,
    shuffle=True,
    subset="validation",
    target_size = (260, 1323)
)

test_dataset = test_datagen.flow_from_directory(
    test_dir,
    #class_mode='binary',
    class_mode='categorical',
    batch_size = 2,
    shuffle=False,
    target_size = (260, 1323)
)


Found 12653 images belonging to 1 classes.
Found 3163 images belonging to 1 classes.
Found 15816 images belonging to 1 classes.


In [5]:
# define model
input_shape = (260, 1323,3)
depth = 20 # For ResNet, specify the depth (e.g. ResNet50: depth=50)
num_classes = 10
attention_module = 'cbam_block'
model = resnet_v1.resnet_v1(input_shape=input_shape, depth=depth, num_classes=num_classes, attention_module=attention_module)
# model = resnet_v2.resnet_v2(input_shape=input_shape, depth=depth, num_classes=num_classes, attention_module=attention_module)   
# model = inception_v3.InceptionV3(input_shape=input_shape, classes=num_classes, attention_module=attention_module)

model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 260, 1323, 3 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 260, 1323, 16 448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 260, 1323, 16 64          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 260, 1323, 16 0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [6]:
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='model_resent20_cbam.png', show_shapes=True)

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


In [7]:
!pip install graphviz pydot -i https://pypi.douban.com/simple

Looking in indexes: https://pypi.douban.com/simple


In [8]:
from keras.optimizers import SGD, Adam
import numpy
sgd = SGD(lr=0.001, decay=1e-7, momentum=.9)
adam = Adam(learning_rate=0.0001)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [9]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import TensorBoard
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy',
                                            patience=1,
                                            verbose=1,
                                            factor=0.25,
                                            min_lr=0.00001)
logs = TensorBoard(log_dir='./logs')

cb_checkpointer = ModelCheckpoint(filepath = 'resnetv1_checkpoint.h5', monitor = 'val_accuracy', save_best_only = True, mode = 'max')

In [ ]:
history = model.fit(train_dataset, validation_data=valid_dataset , epochs = 50, callbacks=[cb_checkpointer, learning_rate_reduction, logs])

Epoch 1/50
   1/6327 [..............................] - ETA: 0s - loss: 31.9332 - accuracy: 0.0000e+00WARNING:tensorflow:From D:\anaconda\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
1767/6327 [=======>......................] - ETA: 19:10 - loss: 1220.0771 - accuracy: 0.0809

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
N = np.arange(0, 10)
plt.style.use('ggplot')
plt.figure()
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.title('Training loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
N = np.arange(0, 10)
plt.style.use('ggplot')
plt.figure()
plt.plot(history.history['accuracy'], label='train_accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.title('Training accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
score = model.evaluate(test_dataset, batch_size=32)
print('Test Loss = ', score[0])
print('Test Accuracy = ', score[1])

In [ ]:
##CONFUSION MATRIX
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
Y_pred = model.predict(test_dataset)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
cnf_matrix = confusion_matrix(test_dataset.classes, y_pred)
print(cnf_matrix)
print('Classification Report')
target_names = ['4asps', '5ADINAC', '6AELA', '7AH', '10PaSC', '11P', '15CT', '16EPA', 'Alcohol_phenol', 'Aldehyde_ketone']
print(classification_report(test_dataset.classes, y_pred, target_names=target_names))